# Python Advanced<img src="python-logo.png" width="150" align="right">
### Toolset 2 - Flask alapok

API-k

Route-olás (https://www.youtube.com/watch?v=8lN4TSslz-0)

Dependency: 
A Flask a WerkZeug WSGI interfészre, a JINJA template engine-re és a Click CLI toolsetre támaszkodik.

WSGI
JINJA
CLICK

telepítsük fel a Flask-ot
(virtuális környezetbe is lehet, sőt)

`pip install Flask`

ellenőrizzük a telepítést: 
`import flask`


Az alábbi példában egy CRUD-képes, User Authenticationt is tartalmazó webalkalmazást készítünk.

1. Létrehozzuk az inicializálásért felelős `__init__.py` fájlt - ez fut majd először a Flask elindításakor. Ebben szerepeljen az import, valamint a create_app függvény, amely példányosítja a Flask appot és visszaadja.
```python
from flask import Flask

def create_app():
    app = Flask(__name__)   
    return app  
```
A Flask (mivel "opinionated") meghatározott folderek és fájlok, osztályok és függvények együttesét követeli meg. Ez ugyan egy szerteágazó struktúra, de pl. az `__init__.py` jó arra, hogy a legtöbb komponens létrehozását csak egyszer csináljuk meg.    
1. Szükség lesz az authentikálásra, majd a jogosult felhasználónál a CRUD tevékenységek ellátására.
2. Ezekhez hozzunk létre két fájl `auth.py` és `main.py` néven.
A `main.py`-ban a Blueprint osztályt és a render_template függvényt importáljuk. 
A Blueprint segítségével rendszerezhetjük, milyen fájlstruktúrával dolgozik a Flask alkalmazásunk. A példában main néven egy Blueprintet csinálunk, ami a 'main.py' nevű fájlt a projekthez rendeli (`__name__`):
```python
main = Blueprint("main", __name__)  
```

Minden Flask függvényt egy dekorátorral látjuk el, amelynek struktúrája pl.:`@app_name.route()`
```python
@main.route('/')
def index():
    return "Hello world"
```

3. Ezt követően az app és a main összekötése is feladat: az app-hoz a main-t (ami, ugye, egy blueprint) kell regisztrálni:
```python
from main import main as main_blueprint
    app.register_blueprint(main_blueprint) 
```
4. Ugyanezeket a lépéseket meg kell tenni az auth.py fájlban (ez lesz a blueprint), illetve ezt is hozzá kell kötni az apphoz az init.py fájlban.

```python

```


5. Ahhoz, hogy futtassuk az alkalmazást, előbb exportálni kell a FLASK_APP környezeti változóba a megfelelő könyvtárat.   
MacOSX és Linux:
```bash
$ export FLASK_APP=...
$ flask run
```

Windows CMD:
```bash
> set FLASK_APP=...
> flask run
```

Windows PowerShell, abból a könyvtárból, amelyen belül a pushups_logger mappa van:   
```PowerShell
> $env:FLASK_APP = "pushups_logger"
> flask run
```

6. Magukat a HTML fájlokat rendereltetni kell, ezt majd a render_template függvény csinálja:
```python
return render_template('index.html')
```
7. A HTML template-eket a __templates__ mappában kell elhelyezni, html kiterjesztéssel. Ezek nem teljes HTML5 leírások (hiányzik belőlük pl. a html tag is), majd a template engine állítja össze a közös és az egyedi részekből a konkrét lapokat.

8. Létrehozzuk a static foldert, ebben lesznek a statikus erőforrások (pl. képek)
9. Ha nem akarjuk sokszor leírni ugyanazt a kódot, vagy csak szeretnénk egy olyan elemet, amely minden html-lapunkon közös (pl. egy oldalsávot navigációs résszel), akkor:
   1.  template inheritance-t használunk majd
   2.  lesz ehhez egy base.html vagy layout.html weblapunk, amely az oldalsávért felel: ebbe a bootstrapet és onnan egy navbart alakítsunk ki;
   3.  egyes részei a template-nek feltölthetők (lennének) statikus tartalommal, de a Flaskban sokkal jobban kedvelik az erre a célra készített kódokat és template-tageket, amit a Jinja kezel.
   4.  a {% és %} közötti részek a Jinja control statement-jei, utasítások vagy a `block` utasítás.
   5.  a {{}} közötti rész változót vagy egyéb python kódot tartalmaz
10. miután a base.html létrejött, nézzük meg a template inheritance működését.
    1.  az egyes html fájlokban a base template-re is hivatkozni kell, meg arra is, ahol felülírjuk az onnan érkező működést, pl. az index.html így néz majd ki:
    ```python
    {% extends "base.html" %} 
    {% block head %} {{ super() }}
    <link
    rel="stylesheet"
    link="{{ url_for('static', filename='extended_beauty.css')}}"
    >
    {% endblock %}
    ```
    Itt fontos megemlíteni, hogy a template-et miből kiindulva (HTML) szerkesztgetjük, mivel egészítjük ki ({{}} és {%%} kódok), hogy ezt ki fogja majd a Flasknak feldolgozni, illetve hova mi teendő. ugyancsak, hogy a website és az alkalmazás struktúrája milyen.
    


11. A következő lépés a signup és login lehetőségek kialakítása, ezzel kapcsolatban az adatbázis kezelése.
12. Mindenekelőtt az API-k és a REST API-kkal kapcsolatos fogalmakat ismerjük meg, ha korábban ez nem lett volna meg, különös tekintettel a POST requestre.
13. A signup template-be berakjuk a formra vonatkozó tartalmat. Itt a 
```python
{ %block content % }
```
részen belül dolgozunk, a header block-ot (a css bootstrap linkekkel együtt) majd a template engine hozzáteszi.
14. A formban a 
```html
<form method="POST" action="/signup">
```
részben két folgot is definiálunk: 1. a gomb megnyomsára POST metódust választunk, 2. hol van az a route, ami ezt feldolgozza (ne feledkezzünk meg a választott változónevekről).
15. Módosítani kell az auth file-ban a signup route-ot. Most válasszuk külön a default GET és az új POST metódus route-jait.
```python
@auth.route('/signup', methods=["POST"])
def signup_post():
    name = request.form.get("name")
    email = request.form.get("email")
    password = request.form.get("password")
    
    print(email, name, password)
    
    return redirect(url_for('auth.login'))
```
16. Most az adatbázisok kezelése következik. Egy jól működő webapp perzisztens adatbázist kezel. Ehhez pl. a SQLite3 megfelel. A kényelmes kezeléshez egy ORM (Object-Relational Mapping) kényelmes, ilyen pl. az SQLAlchemy, ami leképezi az SQL számára szükséges elemeket az objektumstruktúrából.   
SQLAlchemyt nem kínál a flask, de a flask community megcsinálta, a használatához a flask-sqlalchemy-re lesz szükség.
```bash 
python -m pip install flask-sqlalchemy
```

17. Meg kell majd csinálni az SQLite3 adatbázist is, ezt egyszerű egy önálló python scriptből vagy parancssorból elintézni:
```python
import sqlite3

try:
    conn = sqlite3.connect("db.sqlite")
    print("Database connected successfully")
except ConnectionError: 
    print("SQLite table connection error!")

SQL_STM = """
CREATE TABLE IF NOT EXISTS USER
         (ID INT PRIMARY KEY     NOT NULL,
         EMAIL      CHAR(100)    UNIQUE NOT NULL,
         PASSWORD   CHAR(50)     NOT NULL,
         NAME       CHAR(100)
         );
"""
conn.execute(SQL_STM)
print("Table USER created")

conn.close()
```

18. Ahhoz, hogy kényelmesen tudjunk dolgozni Flaskból, az SQLAlchemy-ben először a `model`-t kell létrehozni, ez az adatmodellünk alapja, egy osztály, amely definiálja az egyes adatbázis-táblákkal a kapcsolatot. Egy osztály egy tábla (entitás az ERM-ben), a kapcsolatok, a nyilak később jönnek.   
A fájlt nevezzük el `models.py` néven.
```python
from __init__ import db

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(100), unique=True)
    password = db.Column(db.String(20)) 
    name = db.Column(db.String(100))
```
19. Mivel nem akarjuk eltárolni a jelszót cleartextben, hashelni fogjuk, és csak a hasht tároljuk el.
```python
from werkzeug.security import generate_password_hash
```
20. Mivel az import rendszerben elakadt a példa, próbáljuk meg hagyományos adatbázis-műveletekkel, amelynek a lépései: 
- megnyitjuk az adatbázist
- elvégezzük a mvűveletet
- lezárjuk az adatbázist.
Mindezeket természetesen abban a függvényben, ahol épp szükség van rá.
Ebben a formában ez a nyitás/zárás rendkívüli overhead, később foglalkozunk vele.
A signup függvényben az adatbázisból le kell kérdezni, hogy van-e már megfelelő userünk, ha van, akkor kiírjuk (a console-ra) hogy van már ilyen user, egyébként pedig felviszünk egy újat INSERT INTO SQL utasítással.
21. Most a login funkciót valósítjuk meg a Flask-login segítségével, ami több (nem triviális) szolgáltatást is nyújt:
- véd a cookie-thiefek ellen
- kezeli a Remember Me gombot
- kezeli hosszabb távon is a user sessiont
- lehetővé teszi, hogy egyes oldalrészletek (pl a Profile) csak authentikált userek számára legyenek láthatók.
22. 





